# 实验报告

> 实验1、2、3见对应的ipynb文件

## 实验环境

PC1:
- CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
- OS: Ubuntu 18.04.6
- Others: Python 3.7.15

PC2:
- CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
- OS: kali-linux-2022.3
- Others: Python 3.7.15

实验1、2使用PC1，实验3、4使用PC2

## 数据集
**CIFAR10**

训练集：49000张

验证集：1000张

测试集：1000张

图片尺寸：3 * 32 * 32

## 实验四 网络架构设计
  
要求设计一个 CNN，在 CIFAR10 验证集上准确率超过 60%。由于实验三的3层CNN在1个epoch训练后，在验证集上的准确度已经超过50%。
一般来说，在一定范围内网络越深，准确度越高。所以设计一个4层CNN：再加一层卷积层，卷积核大小为7，如下所示：

![arch](./image/arch.png)

## 实验四 结果

经过10个epoches训练后，在验证集的精度为0.621，达到预期要求，每个epoch的准确度如下图所示：

![acc](./image/acc.png)

## 笔记
### 激活函数

目前常用的激活函数是RELU及其改进型。

### 训练模型时的超参数
主要是需要调 learning rate，如果是如果要固定 learning rate 且手动调参，一般是 0.0001，0.001，0.01，0.1，1 依次尝试，观察 loss 的变化情况逐步调整。
下面给出一段常用自动更新 learning rate 的代码：

```python
optimizer = torch.optim.SGD(
    model.parameters(), lr, momentum=0.9, weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, decay_step=2, gamma=0.9)
```

#### Optimizer

- SGD：标准的梯度下降算法
- Adam

#### momentum
借鉴了物理的惯性现象——当一个球体滚动到山谷时不会立即停下来。可以帮助训练时越过鞍点、局部点。标准的梯度下降算法，更新权重取决于当前梯度大小以及学习率。引入 momentum 后，还需要考虑上一步的梯度。

#### scheduler
可以随着训练的进行，动态调整 learning rate。（一般随着epoch或step为增大，减少 learning rate）

### Batch Normalization
#### What&Why

一个好的 error surface 能让模型在 training 阶段快速收敛。比如说一个崎岖不平的 error surface 相比于碗状的就更加难找到一个最优点。

下面左图是一个例子，W_1 方向变化比较平缓而 W_2 方向比较陡峭。在 固定 learning rate 的情况下，W_2 方向就容易来回震荡无法收敛。而经过 Normalization 后的 error surface 类似右下图，无论在哪个方向都比较容易收敛。

![error surface](./image/bn_1.png)

Normalization 在数学上就是将一个一般正态分布转为标准正态分布。

batch normalization，在计算均值和方差的时候，只需要使用一个batch的数据而不需要整个训练集的数据。而在 testing 阶段，由于输入往往是一个数据，batch size = 1，所以使用的均值是training阶段记录下来的 moving average。

![code](./image/bn_2.png)

#### Where & When
normalization 一般用在激活函数之前，以sigmoid为例，如输入先经过 normalization 的话，就会使得输入分布在0附近，此处梯度较大，不容易出现梯度消失的问题。

使用 batch normalization 时，要注意 batch size 不能太小，否则效果会变差。下图[3] 表明 batch size 小于16时，使用batch normalization的resnet50模型在ImageNet上错误率较高。normalization 的算法有很多，如 Group Normalization[3]、Layer normalization[4]，不过Batch normalization 是目前最常用的一种 normalization 算法。

![Group Normalization](./image/bn_3.png)

参考：

[1] [Sergey Ioffe and Christian Szegedy, "Batch Normalization: Accelerating Deep Network Training by Reducing
Internal Covariate Shift", ICML 2015.](https://arxiv.org/abs/1502.03167)

[2] [Hung-yi Lee,【機器學習2021】類神經網路訓練不起來怎麼辦 (五)： 批次標準化 (Batch Normalization) 簡介](https://arxiv.org/abs/1502.03167)

[3] [Yuxin Wu, Kaiming He. Group Normalization](https://arxiv.org/pdf/1803.08494.pdf)

[4] [Ba, Jimmy Lei, Jamie Ryan Kiros, and Geoffrey E. Hinton. Layer Normalization](https://arxiv.org/pdf/1607.06450.pdf)

### 卷积

以下两个网站可以帮助理解CNN：

1. 动图演示各种卷积运算的过程：
https://github.com/vdumoulin/conv_arithmetic

卷积运算[1] 3 * 3 卷积核，padding = 1：![same_padding_no_strides](./image/same_padding_no_strides.gif)



2. 整个 CNN 过程的可视化：

 https://poloclub.github.io/cnn-explainer/

![cnn_explainer](./image/cnn_explainer.png)
